In [12]:
import pandas as pd
import os

path = os.path.join('..', 'dev', 'db_creation', 'table_making.csv')
df = pd.read_csv(path, index_col=0)

In [13]:
df[['street_number', 'street_name']] = df.qAddress.str.split(' ', n=1, expand=True)
df = df[['street_number', 'street_name', 'qSpecies']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174480 entries, 121399 to 15238
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   street_number  174478 non-null  object
 1   street_name    174478 non-null  object
 2   qSpecies       174480 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 5.3+ MB


In [16]:
df = df.astype({'street_number': 'int64'}).sort_values(['street_name', 'street_number']).dropna()

In [19]:
df.loc[df.street_name == 'san jose ave']

,street_number,street_name,qSpecies
TreeID,,,
39786,50,san jose ave,31
39791,50,san jose ave,35
39780,50,san jose ave,31
39789,50,san jose ave,31
39790,50,san jose ave,275
...,...,...,...
93576,2998,san jose ave,30
93577,2998,san jose ave,80
93574,2998,san jose ave,80


In [38]:
# {street_name: (street_number, qSpecies)}
street = df.loc[df.street_name == 'valencia st'][['street_number', 'qSpecies']]
tree_list = [None] * len(street)
for i, row in enumerate(street.itertuples()):
    street_number = row.street_number
    qSpecies = row.qSpecies
    tree_list[i] = (street_number, qSpecies)

tree_list

[(3, 334),
 (4, 2),
 (4, 2),
 (8, 2),
 (12, 2),
 (12, 2),
 (12, 2),
 (14, 2),
 (14, 2),
 (16, 2),
 (75, 96),
 (75, 96),
 (95, 96),
 (95, 96),
 (98, 2),
 (98, 2),
 (98, 2),
 (98, 2),
 (98, 2),
 (100, 25),
 (101, 26),
 (101, 26),
 (101, 2),
 (101, 26),
 (101, 26),
 (101, 2),
 (101, 2),
 (101, 26),
 (101, 2),
 (101, 26),
 (101, 2),
 (111, 9),
 (115, 9),
 (115, 9),
 (135, 26),
 (135, 99),
 (145, 9),
 (145, 9),
 (147, 9),
 (151, 176),
 (151, 9),
 (155, 9),
 (155, 9),
 (170, 9),
 (170, 9),
 (174, 176),
 (174, 2),
 (175, 9),
 (175, 9),
 (175, 176),
 (175, 9),
 (175, 9),
 (175, 25),
 (175, 9),
 (175, 9),
 (175, 9),
 (175, 9),
 (175, 9),
 (177, 9),
 (177, 9),
 (178, 176),
 (198, 2),
 (199, 2),
 (201, 47),
 (201, 96),
 (201, 47),
 (222, 2),
 (224, 2),
 (224, 2),
 (245, 2),
 (245, 2),
 (245, 2),
 (245, 2),
 (245, 2),
 (250, 166),
 (250, 9),
 (250, 9),
 (250, 9),
 (250, 9),
 (250, 166),
 (250, 166),
 (250, 166),
 (250, 9),
 (250, 166),
 (250, 9),
 (250, 166),
 (250, 2),
 (250, 166),
 (250, 166),
 

In [40]:
street_names = df.street_name.unique().tolist()

tree_dict = dict()

for street_name in street_names:
    trees = df.loc[df.street_name == street_name][['street_number', 'qSpecies']]
    
    tree_list = [None] * len(trees)
    
    for i, row in enumerate(trees.itertuples()):
        street_number = row.street_number
        qSpecies = row.qSpecies
        tree_list[i] = (street_number, qSpecies)
        
    tree_dict.update({street_name: tree_list})

In [43]:
import sys
sys.getsizeof(tree_dict)/1000

73.816

In [131]:
len(tree_dict)

2030

In [165]:

street = tree_dict['valencia st']
trees =  list()
street_num = 1468
search_nearby = False
street_buffer = 20

def bisect(street: list, street_num: int) -> list:
    length = len(street)
    middle = int(length / 2)
    
    # street buffer incase bisection lands on previous address
    if street[middle][0] < street_num - street_buffer:
         return bisect(street[middle:], street_num)
    else:
        return street
    
street = bisect(street, street_num)


for i, address in enumerate(street):
    if address[0] >= street_num:
        if address[0] ==  street_num:
            trees.append(address)
        elif trees:
            break
        elif address[0] > street_num and address[0] < street_num + street_buffer:
            search_nearby = True
            next_address_start = i
            next_address = address[0]
            break

if search_nearby:
    for address in street[next_address_start:]:
        if address[0] == next_address:
            trees.append(address)
        else:
            break
            
    previous_address = street[next_address_start - 1][0]
    if previous_address > street_num - street_buffer: 
        for address in street[next_address_start + 1::-1]:
            if address[0] == previous_address:
                trees.append(address)
            elif address[0] < previous_address:
                break

trees

[(1470, 2), (1465, 67), (1465, 67), (1465, 67)]

In [69]:
a = [0, 1, 2]
start = 1
for i, num in enumerate(a[start:]):
    index = i+start
    print(f'{index=}, {num=}')

index=1, num=1
index=2, num=2


In [93]:
a = [i for i in range(5)]
for i in a[3::-1]:
    print(i)

3
2
1
0


In [136]:
test = pickle.dumps(tree_dict['valencia st'])

In [137]:
%%timeit
pickle.loads(test)

60.4 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [59]:
import pickle
with open('tree_dict_test.pkl', 'wb') as fp:
    pickle.dump(tree_dict, fp)

In [61]:
# need to load into memory first
%%timeit #17.3 ms
with open('tree_dict_test.pkl', 'rb') as fp:
    test_dict = pickle.load(fp)

17.3 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
